In [55]:
import torch
%load_ext autoreload
%autoreload 2

if torch.cuda.is_available():
    PROJECT_PATH = "/notebooks/evaluating-cmr-in-mm/"
else:
    PROJECT_PATH = "/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-mm/"
import sys
sys.path.append(PROJECT_PATH)

import torch
import os
from munch import Munch
import pandas as pd
import argparse
from tqdm import tqdm

torch.set_num_threads(4)
from src.data.dataset import Dataset
from src.models.encoders.clip import CLIP
from src.utils.dataset_preprocessing import load_json_annotations
from src.retrieval.retriever import Retriever
from src.metrics.recall_at_k import recall_at_k
from src.models.relevance_estimators.clip_based import RelevanceEstimator
from src.metrics.dcg import DCG
from src.utils.dataset_preprocessing import save_results_dataframe
from src.utils.utils import get_config_path, get_logger

os.environ["http_proxy"] = "http://devproxy.bloomberg.com:82"
os.environ["https_proxy"] = "http://devproxy.bloomberg.com:82"


dataset = 'coco'
model = 'clip'


CONFIG_PATH = get_config_path(dataset=dataset, model=model)
CONFIG_PATH = '/notebooks/evaluating-cmr-in-mm/config/clip/development_coco_dsp.yaml'

with open(CONFIG_PATH, "rb") as f:
    config = Munch.fromYAML(f)

print(
    "Loading the annotations, preparing the dataset, the model, image embeddings..."
)
json_file = load_json_annotations(config=config)

ds_test_split = Dataset(config=config, split="test", json_file=json_file)

# load the model
if model == "clip":
    model = CLIP(config=config)
elif model == "blip":
    raise NotImplementedError
elif model == "flava":
    raise NotImplementedError
elif model == "beit":
    raise NotImplementedError
elif model == "xvlm":
    raise NotImplementedError
else:
    print(
        "Uknown model, please choose among the following options: clip, blip, flava, beit, xvlm"
    )
# load precomputed image embeddings
img_filenames, img_emb = model.compute_image_embeddings()
print('img_filenames: ', img_filenames[:10])

rel_estimator = RelevanceEstimator(config=config, dataset=ds_test_split)
retriever = Retriever(config=config, model=model)
dcg = DCG(config=config, rel_estimator=rel_estimator)

t2i_queries = []
t2i_targets = []
t2i_retrieved_documents = []
t2i_scores = []
t2i_recalls_at_1 = []
t2i_recalls_at_5 = []
t2i_recalls_at_10 = []
t2i_dcgs = []

print("Text-to-image evaluation...")
for datapoint in tqdm(ds_test_split):
    # get textual query and target
    query = datapoint[0]
    target_filename = datapoint[4]
    print('Query: ', query)
    print('Target: ', target_filename)

    retrieved_documents, scores = retriever.retrieve_top_k(
        query=query, documents=img_emb, documents_names=img_filenames, k=10
    )
    
    # print('retrieved_documents: ', retrieved_documents)

    # metrics:
    # compute recall at k
    # t2i recall: there is only one correct item in the collection, i.e., total_in_collection=1
    t2i_recall_at_1 = recall_at_k(
        target_filename=target_filename,
        retrieved_documents=retrieved_documents,
        k=1,
        total_in_collection=1,
    )
    t2i_recall_at_5 = recall_at_k(
        target_filename=target_filename,
        retrieved_documents=retrieved_documents,
        k=5,
        total_in_collection=1,
    )
    t2i_recall_at_10 = recall_at_k(
        target_filename=target_filename,
        retrieved_documents=retrieved_documents,
        k=10,
        total_in_collection=1,
    )
    # print('t2i: recalls at 1, 5, 10: ', t2i_recall_at_1, t2i_recall_at_5, t2i_recall_at_10)

    t2i_dcg = dcg.compute_dcg(
        query=query,
        target_filename=target_filename,
        retrieved_documents=retrieved_documents,
    )
    # print('T2i_dcg: ', t2i_dcg)

    t2i_queries.append(query)
    t2i_targets.append(target_filename)
    t2i_retrieved_documents.append(retrieved_documents)
    t2i_scores.append(scores)
    t2i_recalls_at_1.append(t2i_recall_at_1)
    t2i_recalls_at_5.append(t2i_recall_at_5)
    t2i_recalls_at_10.append(t2i_recall_at_10)
    t2i_dcgs.append(t2i_dcg)

    if datapoint[-1] > 0 and datapoint[-1] % 10 == 0:
        print(f"Progress: {datapoint[-1]}/{len(ds_test_split)}")
        break



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loading the annotations, preparing the dataset, the model, image embeddings...
Loaded annotations from  /notebooks/evaluating-cmr-in-mm/datasets/coco/annotations/dataset_coco.json
Image embeddigns are already precomputed
Loaded precomputed filenames and embeddings from  /notebooks/evaluating-cmr-in-mm/datasets/coco/coco-img-embeddings.pkl
img_filenames:  ['COCO_val2014_000000265550.jpg', 'COCO_val2014_000000106073.jpg', 'COCO_val2014_000000109340.jpg', 'COCO_val2014_000000150235.jpg', 'COCO_val2014_000000099747.jpg', 'COCO_val2014_000000286813.jpg', 'COCO_val2014_000000146457.jpg', 'COCO_val2014_000000493923.jpg', 'COCO_val2014_000000378515.jpg', 'COCO_val2014_000000377626.jpg']
Text-to-image evaluation...


  0%|          | 0/25010 [00:00<?, ?it/s]

Query:  A man with a red helmet on a small moped on a dirt road. 
Target:  COCO_val2014_000000391895.jpg


  0%|          | 1/25010 [00:00<4:13:48,  1.64it/s]

Query:  Man riding a motor bike on a dirt road on the countryside.
Target:  COCO_val2014_000000391895.jpg


...

Query:  A little girl is getting ready to blow out a candle on a small dessert.
Target:  COCO_val2014_000000060623.jpg


  0%|          | 10/25010 [00:03<2:09:34,  3.22it/s]

Query:  A man on a bicycle riding next to a train
Target:  COCO_val2014_000000483108.jpg


  0%|          | 10/25010 [00:03<2:37:07,  2.65it/s]

Progress: 10/25010


In [38]:
%load_ext autoreload
%autoreload 2

In [41]:
from src.utils.dataset_preprocessing import get_img_filenames_full_path, get_img_filenames, divide_chunks, get_precomputed_embeddings_path, load_filenames_embs_from_pkl, get_emb_file_path, dump_filenames_embs_to_pkl



print('Computing image embeddings...')
# generate full path to images
img_root = os.path.join(
    config.dataset.root,
    config.dataset.img_folder)
images_full_paths = get_img_filenames_full_path(img_root=img_root)
img_filenames = get_img_filenames(img_root=img_root)
print('len(img_filenames): ', len(img_filenames))

Computing image embeddings...
len(img_filenames):  40504


In [48]:
capt_ids, capts, capt_embs = model.compute_caption_embeddings(
        ds_split=ds_test_split
    )

Computing caption embeddings...


Batches: 100%|██████████| 196/196 [00:03<00:00, 52.12it/s]


Saving captions...
Saved files to  /notebooks/evaluating-cmr-in-mm/datasets/coco/coco-capt-embeddings.pkl


In [49]:
len(capt_ids)

25010

In [47]:
from src.models.encoders.clip import CLIP

img_filenames, img_emb = model.compute_image_embeddings()

Computing image embeddings...
len(img_filenames):  40504
Progress: 0/41


Batches:  25%|██▌       | 2/8 [00:03<00:11,  1.87s/it]


KeyboardInterrupt: 

In [37]:
from src.models.encoders.clip import CLIP

config

Munch({'dataset': Munch({'root': '/notebooks/evaluating-cmr-in-mm/datasets/coco/', 'img_folder': 'val2014', 'img_emb_filename': 'coco-img-embeddings.pkl', 'capt_emb_filename': 'coco-capt-embeddings.pkl', 'annotation_file': 'annotations/dataset_coco.json'}), 'model': Munch({'name': 'clip-ViT-B-32', 'path': 'TBA', 'max_seq_length': 330}), 'dcg': Munch({'relevance_estimator': Munch({'name': 'clip-ViT-B-32', 'sim_score': 'cosine'})}), 'results': Munch({'dir': '/notebooks/evaluating-cmr-in-mm/results'})})

In [43]:
len([x for x in img_filenames if 'train' in x])

0

In [44]:
len([x for x in img_filenames if 'test' in x])

0

In [45]:
len([x for x in img_filenames if 'val' in x])

40504

In [46]:
len(img_filenames)

40504

In [4]:

data = {
    "t2i_queries": t2i_queries,
    "t2i_targets": t2i_targets,
    "t2i_retrieved_documents": t2i_retrieved_documents,
    "t2i_scores": t2i_scores,
    "t2i_recalls_at_1": t2i_recalls_at_1,
    "t2i_recalls_at_5": t2i_recalls_at_5,
    "t2i_recalls_at_10": t2i_recalls_at_10,
    "t2i_dcgs": t2i_dcgs,
}

    t2i_results = pd.DataFrame(data=data)

    print(t2i_results.describe())
    print(t2i_results)

    logging.info("Saving the results...")
    save_results_dataframe(
        config=config, dataf=t2i_results, root=f'{args.model}/{args.dataset}', filename=f"t2i-results"
    )

    print("Done!")

In [2]:
!pwd

/notebooks/evaluating-cmr-in-mm


In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('../')

In [6]:
import argparse
from munch import Munch

class Args(argparse.Namespace):
  perturbation = 'none'
args=Args()

config.args = Munch(
    perturbation=args.perturbation
)

In [3]:
from src.utils.dataset_preprocessing import get_precomputed_embeddings_path
from src.utils.utils import get_config

path = 'evaluating-cmr-in-mm/datasets/f30k/precomputed/altclip/none/f30k-img-embeddings.pkl'


config = get_config('f30k', 'altclip')

/layers/com.bloomberg.ds.buildpacks.pip/requirements/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Config path:  ./config/altclip/f30k/development_dsp.yaml
Loaded config path


In [10]:
get_precomputed_embeddings_path(config, dtype='img')

'/notebooks/evaluating-cmr-in-mm/datasets/f30k/precomputed/altclip/none/f30k-img-embeddings.pkl'